In [1]:
'''
This code uses Lazy Predict model.
Lazy Predict is a Python library that automates the machine learning model
building process. It quickly analyzes data and fits multiple machine learning
models to identify top candidates based on performance metrics. This allows
users to get an initial understanding of the potential performance of different
machine learning algorithms on their datasets without manually tuning each
individual model.
'''

'\nThis code uses Lazy Predict model.\nLazy Predict is a Python library that automates the machine learning model\nbuilding process. It quickly analyzes data and fits multiple machine learning\nmodels to identify top candidates based on performance metrics. This allows\nusers to get an initial understanding of the potential performance of different\nmachine learning algorithms on their datasets without manually tuning each\nindividual model.\n'

In [2]:
!pip install lazypredict

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 MB 3.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-runtime-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cudnn-cu12==8.9.2.26; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requ

In [3]:

import os
import math
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_predict, train_test_split, GridSearchCV
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    accuracy_score,
    classification_report,
    r2_score,
    mean_absolute_percentage_error
)
from sklearn.ensemble import IsolationForest

from lazypredict.Supervised import LazyRegressor


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [4]:
# ********************** FUNCTION DEFINITIONS **********************

def check_and_get_filename(directory, base_name, extension):
    """
    Check if the specific file already exists and create a new name if necessary.
    """
    count = 0
    while True:
        file_name = f"{base_name}{'' if count == 0 else f'_{count}'}.{extension}"
        full_path = os.path.join(directory, file_name)
        if not os.path.exists(full_path):
            return full_path
        count += 1

def save_to_file_and_print(message, file):
    """Prints a message and appends it to a file."""
    try:
        print(message)
        with open(file, 'a') as f:
            f.write(message)
    except Exception as e:
        print(f"An error occurred while saving the message to the file: {e}")


def format_value(value, decimal_places):
    format_string = "{:." + str(decimal_places) + "f}"
    return float(format_string.format(value))


def calculate_metrics(phase_name, y_true, y_pred, metrics_to_print=None, file=None):
    '''
    Calculates metrics.
    If no specific metrics are requested (metrics_to_print=None), it calculates
    and includes all available metrics in the message. Otherwise, it only
    calculates the necessary ones.
    The message is saved in a file if requested (file = 'name')
    '''
    if metrics_to_print is None:
        metrics_to_print = ['MAPE', 'MAE', 'MSE', 'RMSE', 'R2']

    metric_functions = {
        'MAPE': lambda y_true, y_pred: mean_absolute_percentage_error(y_true, y_pred),
        'MAE': mean_absolute_error,
        'MSE': mean_squared_error,
        'RMSE': lambda y_true, y_pred: math.sqrt(mean_squared_error(y_true, y_pred)),
        'R2': r2_score
    }

    output_message = f'\nMetrics for {phase_name} phase:\n'

    for metric in metrics_to_print:
        value = metric_functions[metric.upper()](y_true, y_pred)
        if metric.upper() == 'MAPE':
            formatted_value = format_value(value, 2)
            formatted_value = f'{formatted_value*100}%'
        else:
            formatted_value = format_value(value, 6)
        output_message += f"{metric}: {formatted_value}\n"

    if file:
        save_to_file_and_print(output_message, file)

    return output_message


In [15]:

# ********************** PARAMETER DEFINITIONS **********************
property = "Cp (J/mol*K)"
property_name = property.split(' (')[0]

'''
"Density (kg/m3)","Volume (m3/kg)",
"Internal Energy (kJ/mol)","Enthalpy (kJ/mol)","Entropy (J/mol*K)",
"Cv (J/mol*K)","Cp (J/mo*lK)","Sound Spd. (m/s)","Joule-Thomson (K/MPa)",
"Viscosity (uPas)","Therm. Cond. (W/m*K)","Phase"
'''

## files and directories
#current_directory = os.getcwd()    #if used on the notebook
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


KeyboardInterrupt: 

In [6]:
current_directory = f'gdrive/MyDrive/Thermophysical/{property_name}'

In [7]:

# ********************** DATA ACQUISITION **********************
                     #*****  NIST  *****
            # **** Temperature, Pressure, Phase ****
'''
In this section, data from NIST is acquired for Temperature, Pressure, and the
corresponding physical state. These data are solely used to train a decision
tree to determine the phase in which the fluid exists at a given pressure and
temperature.
'''
NIST_directory = os.path.join(current_directory, "NIST")
txt_files_NIST = [file for file in os.listdir(NIST_directory) if file.endswith(".txt")]

X_NIST = []
state_column = []

for file in txt_files_NIST:
  df_NIST = pd.read_csv(os.path.join(NIST_directory, file), delimiter='\t')
  X_NIST_file = df_NIST[["Temperature (K)","Pressure (MPa)"]].values
  state_column_file = df_NIST.iloc[:, -1]  # Assuming the last column indicates the state

  X_NIST.extend(X_NIST_file)
  state_column.extend(state_column_file)

X_NIST = np.array(X_NIST)

# ***** DECISION TREE *****
DT_file = check_and_get_filename(current_directory, "DecisionTree_Physical-State-Determination_LazyPredict", "log")

## Mapping of physical states. Associate them with numbers (0, 1, 2) for use in the decision tree
state_mapping = {'liquid': 0, 'vapor': 1, 'supercritical': 2}
state_mapped = [state_mapping[state] for state in state_column]

# Spliting the data into training (70%) and testing (30%) sets (better than 80-20)
X_train, X_test, state_train, state_test = train_test_split(X_NIST, state_mapped, test_size=0.3, random_state=42)

log_info = f"Decision tree to predict physical states based on temperature and pressure inputs.\nNumber of data: {len(X_NIST)}\nNumber of train data: {len(X_train)}\nNumber of test data: {len(X_test)}"
save_to_file_and_print(log_info, DT_file)

decision_tree = DecisionTreeClassifier(random_state=42)

decision_tree.fit(X_train, state_train)

state_pred = decision_tree.predict(X_test)

accuracy_tree = accuracy_score(state_test, state_pred)

save_to_file_and_print(f"\nAccuracy of Decision Tree model: {accuracy_tree * 100:.3f}%", DT_file)
save_to_file_and_print(classification_report(state_test, state_pred), DT_file)

# Metrics - Decision Tree Evaluation
save_to_file_and_print("\n\n > Decision Tree Evaluation <", DT_file)
calculate_metrics('Decision Tree', state_test, state_pred, ['RMSE', 'R2'], DT_file)




Decision tree to predict physical states based on temperature and pressure inputs.
Number of data: 4991983
Number of train data: 3494388
Number of test data: 1497595

Accuracy of Decision Tree model: 99.998%
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85252
           1       1.00      1.00      1.00      7157
           2       1.00      1.00      1.00   1405186

    accuracy                           1.00   1497595
   macro avg       1.00      1.00      1.00   1497595
weighted avg       1.00      1.00      1.00   1497595



 > Decision Tree Evaluation <

Metrics for Decision Tree phase:
RMSE: 0.004834
R2: 0.999893



'\nMetrics for Decision Tree phase:\nRMSE: 0.004834\nR2: 0.999893\n'

In [13]:


# ********************** DATA ACQUISITION **********************
            # **** Temperature, Pressure, Property ****
print(f"\n\n*************Start analysis of property '{property_name}'*************")

property_directory = os.path.join(current_directory, property_name)
Experimental_directory = os.path.join(property_directory, "Experimental")


# ********************** LazyPredict **********************
LazyPredict_directory = os.path.join(property_directory, "LazyPredict")
os.makedirs(LazyPredict_directory, exist_ok=True)
LazyPredict_file = check_and_get_filename(LazyPredict_directory, "LazyPredict", "log")


# *****  EXPERIMENTAL  *****
txt_files_Exp = [file for file in os.listdir(Experimental_directory) if file.endswith(".csv")]
y_Exp = []  ## Dependent variable for all states
X_Exp = []  ## Independent variables (Temperature and Pressure) for all states




*************Start analysis of property 'Cp'*************


In [16]:

for file_exp in txt_files_Exp:
    df_Exp = pd.read_csv(os.path.join(Experimental_directory, file_exp), delimiter=',')
    X_Exp_file = df_Exp[["Temperature (K)","Pressure (MPa)"]].values
    y_Exp_file = df_Exp[property].values
    X_Exp.extend(X_Exp_file)
    y_Exp.extend(y_Exp_file)

X_Exp = np.array(X_Exp)
X_Exp=X_Exp.tolist()
if property == "Therm. Cond. (W/m*K)":
    y_Exp = [x * 1000 for x in  y_Exp]
else:
    pass
y_Exp = [[x] for x in  y_Exp]

print(X_Exp)

[[294.26, 0.0], [294.26, 1.72], [294.26, 3.45], [294.26, 5.17], [294.26, 6.89], [294.26, 8.62], [294.26, 10.34], [310.93, 0.0], [310.93, 1.72], [310.93, 3.45], [310.93, 5.17], [310.93, 6.89], [310.93, 8.62], [310.93, 10.34], [327.59, 0.0], [327.59, 1.72], [327.59, 3.45], [327.59, 5.17], [327.59, 6.89], [327.59, 8.62], [327.59, 10.34], [344.26, 0.0], [344.26, 1.72], [344.26, 3.45], [344.26, 5.17], [344.26, 6.89], [344.26, 8.62], [344.26, 10.34], [360.93, 0.0], [360.93, 1.72], [360.93, 3.45], [360.93, 5.17], [360.93, 6.89], [360.93, 8.62], [360.93, 10.34], [377.59, 0.0], [377.59, 1.72], [377.59, 3.45], [377.59, 5.17], [377.59, 6.89], [377.59, 8.62], [377.59, 10.34], [409.82, 5.07], [409.82, 5.07], [409.82, 5.07], [365.93, 5.07], [365.93, 5.07], [365.93, 5.07], [365.93, 5.07], [343.15, 5.07], [343.15, 5.07], [343.15, 5.07], [334.82, 5.07], [334.26, 5.07], [332.59, 5.07], [121.1, 5.0], [121.1, 5.0], [121.1, 5.0], [121.0, 5.0], [136.1, 5.0], [136.2, 5.0], [136.2, 5.0], [151.3, 5.0], [151.3,

In [17]:

# *****  NIST  *****
'''
NIST data that presents the same temperature and pressure conditions as those
used in the experimental data has been acquired.
'''
NIST_Expbased_directory = os.path.join(NIST_directory, "results_experimental-based")
txt_files_NIST_Expbased = [file for file in os.listdir(NIST_Expbased_directory) if file.endswith(".txt")]

y_NIST_Expbased = []  ## Dependent variable for all states
X_NIST_Expbased = []  ## Independent variables (Temperature and Pressure) for all states

for file_exp in txt_files_NIST_Expbased:
  df_NIST_Expbased = pd.read_csv(os.path.join(NIST_Expbased_directory, file_exp), delimiter='\t')
  X_NIST_Expbased_file = df_NIST_Expbased[["Temperature (K)","Pressure (MPa)"]].values
  y_NIST_Expbased_file = df_NIST_Expbased[property].values
  X_NIST_Expbased.extend(X_NIST_Expbased_file)
  y_NIST_Expbased.extend(y_NIST_Expbased_file)


'''
Since y_NIST_Expbased is a scalar value and X_NIST_Expbased is a NumPy array (array([T,P])),
They cannot be accessed using indexing.
One possible solution is to convert it into a list.
'''
X_NIST_Expbased = np.array(X_NIST_Expbased)
X_NIST_Expbased=X_NIST_Expbased.tolist()
if property == "Therm. Cond. (W/m*K)":
  y_NIST_Expbased = [x * 1000 for x in  y_NIST_Expbased]
else:
  pass
y_NIST_Expbased = [[x] for x in  y_NIST_Expbased]

In [18]:

'''
Given that not all experimental data falls within the NIST range of Temperature
and pressure, the values that lie within the range are stored in a variable for
error analysis.
'''

missing_data_X = []
matched_data = []

for x_exp, y_exp in zip(X_Exp, y_Exp):
    if any(np.array_equal(x_exp, x_nist) for x_nist in X_NIST_Expbased):
        matched_data.append((x_exp, y_exp))
    else:
        missing_data_X.append(x_exp)

missing_data_y = [y for x, y in zip(X_Exp, y_Exp) if any(np.array_equal(x, x_nist) for x_nist in missing_data_X)]

matched_data_X = [x for x, _ in matched_data]
matched_data_y = [y for _, y in matched_data]

save_to_file_and_print(f"Number of matched data between NIST and Experimental: {len(matched_data_X)}\nNumber of missing data between NIST and Experimental: {len(missing_data_X)}", LazyPredict_file)

X_liquid_Exp = []
y_liquid_Exp = []
X_vapor_Exp = []
y_vapor_Exp = []
X_supercritical_Exp = []
y_supercritical_Exp = []


Number of matched data between NIST and Experimental: 513
Number of missing data between NIST and Experimental: 0


In [19]:
#**** Discovering the physical state in the studied condition and separating the set****
#{'liquid': 0, 'vapor': 1, 'supercritical': 2}
state_pred_list_Exp_Expbased = decision_tree.predict(matched_data_X)

#Convert to a list type:
state_pred_list_Exp_Expbased = [[x] for x in state_pred_list_Exp_Expbased]


for i in range(len(state_pred_list_Exp_Expbased)):

  if state_pred_list_Exp_Expbased[i] == [0]:  # Liquid state
    X_liquid_Exp.append(matched_data_X[i])
    y_liquid_Exp.append(matched_data_y[i])
  elif state_pred_list_Exp_Expbased[i] == [1]:  # Vapor state
    X_vapor_Exp.append(matched_data_X[i])
    y_vapor_Exp.append(matched_data_y[i])
  elif state_pred_list_Exp_Expbased[i] == [2]:  # Supercritical state
    X_supercritical_Exp.append(matched_data_X[i])
    y_supercritical_Exp.append(matched_data_y[i])

print(f"Number of data points in Liquid state: {len(X_liquid_Exp)}")
print(f"Number of data points in Vapor state: {len(X_vapor_Exp)}")
print(f"Number of data points in Supercritical state: {len(X_supercritical_Exp)}")



Number of data points in Liquid state: 195
Number of data points in Vapor state: 164
Number of data points in Supercritical state: 154


In [20]:

#===========================================
#               LAZY PREDICT
#===========================================
save_to_file_and_print("\n\n>>> LIQUID PHASE <<<\n", LazyPredict_file)





>>> LIQUID PHASE <<<



In [ ]:

#           >>>>>       LIQUID PART     <<<<<
# Spliting the data into training (80%) and testing (20%) sets for liquid state
X_liquid_train, X_liquid_test, y_liquid_train, y_liquid_test = train_test_split(X_liquid_Exp, y_liquid_Exp, test_size=0.2, random_state=42)

X_liquid_train_array = np.array(X_liquid_train)
X_liquid_test_array = np.array(X_liquid_test)
y_liquid_train_array = np.array(y_liquid_train)
y_liquid_test_array = np.array(y_liquid_test)

reg_liquid = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
models_liquid,predictions = reg_liquid.fit(X_liquid_train_array, X_liquid_test_array, y_liquid_train_array, y_liquid_test_array)

save_to_file_and_print(f"\nResults from lazy predict:\n {models_liquid}.\n", LazyPredict_file)

save_to_file_and_print("\n\n>>> VAPOR PHASE <<<\n", LazyPredict_file)



 79%|███████▊  | 33/42 [00:01<00:00, 32.70it/s]

QuantileRegressor model failed to execute
Solver interior-point is not anymore available in SciPy >= 1.11.0.


 88%|████████▊ | 37/42 [00:01<00:00, 26.94it/s]

In [ ]:
#           >>>>>       VAPOR PART     <<<<<
# Spliting the data into training (80%) and testing (20%) sets for vapor state
X_vapor_train, X_vapor_test, y_vapor_train, y_vapor_test = train_test_split(X_vapor_Exp, y_vapor_Exp, test_size=0.2, random_state=42)

X_vapor_train_array = np.array(X_vapor_train)
X_vapor_test_array = np.array(X_vapor_test)
y_vapor_train_array = np.array(y_vapor_train)
y_vapor_test_array = np.array(y_vapor_test)

reg_vapor = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
models_vapor,predictions = reg_vapor.fit(X_vapor_train_array, X_vapor_test_array, y_vapor_train_array, y_vapor_test_array)

save_to_file_and_print(f"\nResults from lazy predict:\n {models_vapor}.\n", LazyPredict_file)


In [ ]:
print(f'X train: {X_vapor_train_array}\n')
print(f'X test: {X_vapor_test_array}\n')
print(f'y train: {y_vapor_train_array}\n')
print(f'y test: {y_vapor_test_array}\n')

In [ ]:

save_to_file_and_print("\n\n>>> SUPERCRITICAL PHASE <<<\n", LazyPredict_file)
#           >>>>>       SUPERCRITICAL PART     <<<<<
# Spliting the data into training (80%) and testing (20%) sets for liquid state
X_supercritical_train, X_supercritical_test, y_supercritical_train, y_supercritical_test = train_test_split(X_supercritical_Exp, y_supercritical_Exp, test_size=0.2, random_state=42)

X_supercritical_train_array = np.array(X_supercritical_train)
X_supercritical_test_array = np.array(X_supercritical_test)
y_supercritical_train_array = np.array(y_supercritical_train)
y_supercritical_test_array = np.array(y_supercritical_test)

reg_supercritical = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
models_supercritical,predictions = reg_supercritical.fit(X_supercritical_train_array, X_supercritical_test_array, y_supercritical_train_array, y_supercritical_test_array)

save_to_file_and_print(f"\nResults from lazy predict:\n {models_supercritical}.\n", LazyPredict_file)
